In [1]:
from pulp import *
import numpy as np

In [4]:
A = np.array([[3,1,2],[1,3,0],[0,2,4]])
c = np.array([150,200,300])
b = np.array([60,36,48])
(m,n) = A.shape
prob = LpProblem(name='Production', sense=LpMaximize)
x = [LpVariable('x'+str(i+1), lowBound=0) for i in range(n)]
prob += lpDot(c,x)
for i in range(m):
  prob += lpDot(A[i],x) <= b[i], 'ineq'+str(i)

print(prob)

prob.solve()
print(LpStatus[prob.status])
print('Optimal value =', value(prob.objective))
for v in prob.variables():
  print(v.name, '=', v.varValue)

Production:
MAXIMIZE
150*x1 + 200*x2 + 300*x3 + 0
SUBJECT TO
ineq0: 3 x1 + x2 + 2 x3 <= 60

ineq1: x1 + 3 x2 <= 36

ineq2: 2 x2 + 4 x3 <= 48

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/0f7183a839c04492b7119514b1adc961-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/0f7183a839c04492b7119514b1adc961-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 19 RHS
At line 23 BOUNDS
At line 24 ENDATA
Problem MODEL has 3 rows, 3 columns and 7 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 3 (0) columns and 7 (0) elements
0  Obj -0 Dual inf 650 (3)
0  Obj -0 Dual inf 650 (3)

In [5]:
X = np.array([v.varValue for v in prob.variables()])
np.all(np.abs(b - np.dot(A,X)) <= 1.0e-5)

True

In [6]:
AT = A.T
dual = LpProblem(name='Dual_Production', sense=LpMinimize)
y = [LpVariable('y'+str(i+1), lowBound=0) for i in range(m)]
dual += lpDot(b,y)
for j in range(n):
  dual += lpDot(AT[j],y) >= c[j], 'ineq'+str(j)

dual.solve()
print(LpStatus[dual.status])
print('Optimal value of dual problem =', value(dual.objective))
for v in dual.variables():
  print(v.name, '=', v.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/58468bb83fc0485fa223c5092fb90e27-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/58468bb83fc0485fa223c5092fb90e27-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 19 RHS
At line 23 BOUNDS
At line 24 ENDATA
Problem MODEL has 3 rows, 3 columns and 7 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 3 (0) columns and 7 (0) elements
0  Obj 0 Primal inf 191.66667 (3)
3  Obj 5800
Optimal - objective value 5800
Optimal objective 5800 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01



In [7]:
Y =np.array([v.varValue for v in dual.variables()])
np.all(np.abs(np.dot(AT,Y) - c) <= 1.0e-5)

True